# Importing libraries, loading and transforming data

In [1]:
!pip install -q evaluate transformers==4.28.1
!pip install -U -q datasets
!pip install -q torchaudio==0.12
!add-apt-repository -y ppa:savoury1/ffmpeg4 
!apt-get -qq install -y ffmpeg
!pip install -q mlflow

ERROR: Ignored the following yanked versions: 2.0.0
ERROR: Could not find a version that satisfies the requirement torchaudio==0.12 (from versions: 2.0.1, 2.0.2, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0)
ERROR: No matching distribution found for torchaudio==0.12
zsh:1: command not found: add-apt-repository
zsh:1: command not found: apt-get


In [2]:
#imports
import pandas as pd
import gc
import re
import numpy as np

import warnings 
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import torch
import datasets
import transformers
print(transformers.__version__)

4.28.1


In [3]:
df_list = []
for loc in ['Train', 'Test']:
    df_tmp = pd.read_csv(f"downloads/Metadata_{loc}.csv")
    df_list.append(df_tmp)
df = pd.concat(df_list, axis=0)
df['Class'] = df['Class'].replace({"Sound_Guiatr": "Sound_Guitar"})
# remove violin label as it is contaminated by Drums
df = df[df['Class']!="Sound_Violin"]
print(df.shape)
df.sample(5).T

(1989, 2)


,2478,521,1027,862,110
FileName,ROOM_room6_MUS_beethoven2_DEV_stereomic.wav,LP_Lick1_MN.wav,WaveDrum02_39KD (123).wav,freedom-in-sight-130bpm-32759.wav,classical_2_60BPM.wav
Class,Sound_Piano,Sound_Guitar,Sound_Drum,Sound_Drum,Sound_Guitar


In [4]:
df['Class'].value_counts()

Class
Sound_Guitar    720
Sound_Drum      720
Sound_Piano     549
Name: count, dtype: int64

In [5]:
RATE_HZ = 16000 # resampling rate in Hz
MAX_LENGTH = 240000 # maximum audio interval length to consider (= RATE_HZ * SECONDS)
labels = ['Sound_Guitar', 'Sound_Drum', 'Sound_Piano']
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

print(id2label, '\n\n', label2id)

{0: 'Sound_Guitar', 1: 'Sound_Drum', 2: 'Sound_Piano'} 

 {'Sound_Guitar': 0, 'Sound_Drum': 1, 'Sound_Piano': 2}


In [6]:
from pathlib import Path
import torchaudio

def load_data():
    file_list = []
    sample_list = []
    for file in Path('./downloads/').glob('*_submission/*_submission/*.wav'):
        sample_name = file.stem + '.wav' #re.match(r"(\d+)", file.stem).group(0)
        file_list.append(file)
        sample_list.append(str(sample_name))
    dd = pd.DataFrame()
    dd['file'] = file_list
    dd['FileName'] = sample_list
    return dd

In [7]:
dd = load_data()
dd = dd.set_index('FileName').join(df.set_index('FileName'), how='inner')
dd = dd[dd['Class'].isin(labels)]
dd['label'] = dd['Class'].apply(lambda x: label2id[x])
selected_cols = ['file', 'label', 'Class']
dd = dd[selected_cols]
if not dd.empty:
    sampled_data = dd.sample(5).T
else:
    print("No data available for sampling.")

In [8]:
dd = dd.reset_index(drop=True)
dd.head()

,file,label,Class
0,downloads/Train_submission/Train_submission/FS...,0,Sound_Guitar
1,downloads/Train_submission/Train_submission/Wa...,1,Sound_Drum
2,downloads/Train_submission/Train_submission/06...,1,Sound_Drum
3,downloads/Train_submission/Train_submission/dr...,1,Sound_Drum
4,downloads/Train_submission/Train_submission/FS...,0,Sound_Guitar


In [10]:
def get_transform_audio(file):
    audio, rate = torchaudio.load(file)
    transform = torchaudio.transforms.Resample(rate, RATE_HZ)
    audio = transform(audio).squeeze(0).numpy()
    audio = audio[:MAX_LENGTH]
    if audio.shape[0] == MAX_LENGTH:
        return audio
dd['audio'] = dd['file'].progress_apply(get_transform_audio)

100%|██████████| 1989/1989 [00:17<00:00, 113.79it/s]


In [12]:
dd = dd.dropna(subset=['audio'])
dd.shape

<class 'pandas.core.frame.DataFrame'>
Index: 848 entries, 4 to 1986
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   file    848 non-null    object
 1   label   848 non-null    int64 
 2   Class   848 non-null    object
 3   audio   848 non-null    object
dtypes: int64(1), object(3)
memory usage: 33.1+ KB


In [13]:
%%time
# random oversampling of all minority classes
y = dd[['label']]
dd = dd.drop(['label'], axis=1)
ros = RandomOverSampler(random_state=83)
dd, y_resampled = ros.fit_resample(dd, y)
del y
dd['label'] = y_resampled
del y_resampled
gc.collect()

CPU times: user 61.8 ms, sys: 68.9 ms, total: 131 ms
Wall time: 261 ms


60

In [14]:
dd.shape, dd['label'].value_counts()

((1095, 4),
 label
 0    365
 2    365
 1    365
 Name: count, dtype: int64)

In [15]:
%%time
dd = dd.drop(['file'], axis=1)

CPU times: user 650 µs, sys: 237 µs, total: 887 µs
Wall time: 992 µs


In [16]:
dd.sample(5).T

,522,127,357,596,167
Class,Sound_Piano,Sound_Guitar,Sound_Piano,Sound_Guitar,Sound_Piano
audio,"[-1.0979224e-05, 2.657303e-06, 3.635023e-06, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-8.767109e-06, 1.5560276e-05, -7.99098e-06, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-1.8003049e-06, 1.3579844e-05, 1.0647742e-05,..."
label,2,0,2,0,2


In [17]:
from datasets import Dataset
dd = Dataset.from_pandas(dd)

In [18]:
from collections import Counter
Counter(dd['label']).items()

dict_items([(0, 365), (2, 365), (1, 365)])

In [19]:
dd = dd.train_test_split(test_size=0.2)
dd

DatasetDict({
    train: Dataset({
        features: ['Class', 'audio', 'label'],
        num_rows: 876
    })
    test: Dataset({
        features: ['Class', 'audio', 'label'],
        num_rows: 219
    })
})

# Load facebook/wav2vec2-base-960h model

In [20]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

model_str = "facebook/wav2vec2-base-960h" 
feature_extractor = AutoFeatureExtractor.from_pretrained(model_str)
model = AutoModelForAudioClassification.from_pretrained(model_str,num_labels=len(labels))
model.config.id2label = id2label
# number of trainable parameters
print(model.num_parameters(only_trainable=True)/1e6)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForSequenceClassification: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.weight', 'projector.weight', 'projector.bias', 'classifier.bias', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be 

94.569347


In [21]:
def preprocess_function(batch):    
    inputs = feature_extractor(batch['audio'], sampling_rate=RATE_HZ, max_length=MAX_LENGTH, truncation=True)
    inputs['input_values'] = inputs['input_values'][0]
    return inputs

dd['test'] = dd['test'].map(preprocess_function, remove_columns="audio", batched=False)
dd['train'] = dd['train'].map(preprocess_function, remove_columns="audio", batched=False)

Map: 100%|██████████| 876/876 [00:48<00:00, 18.00 examples/s]


In [22]:
gc.collect()

32

# Train and evaluate model

In [23]:
import evaluate

accuracy = evaluate.load("accuracy")

from sklearn.metrics import roc_auc_score
def compute_metrics(eval_pred):
    # Compute the ROC AUC score
    predictions = eval_pred.predictions
    predictions = np.exp(predictions)/np.exp(predictions).sum(axis=1, keepdims=True)
    label_ids = eval_pred.label_ids
    roc_auc = roc_auc_score(label_ids, predictions, average='macro', multi_class='ovr') # one-vs-rest ROC AUC score
    
    # Calculate accuracy using the loaded accuracy metric
    acc_score = accuracy.compute(predictions=predictions.argmax(axis=1), references=label_ids)['accuracy']
    
    return {
        "roc_auc": roc_auc,
        "accuracy": acc_score
    }

In [24]:
from transformers import TrainingArguments, Trainer
batch_size=1
warmup_steps=50
weight_decay=0.02
num_train_epochs=2
model_name = "musical_instrument_detection"
training_args = TrainingArguments(
    output_dir=model_name,
    logging_dir='./logs',
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=5e-6, # 1e-6
    logging_strategy='steps',
    logging_first_step=True,
    load_best_model_at_end=True,
    logging_steps=1,
    evaluation_strategy='epoch',
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    gradient_accumulation_steps=4, 
    gradient_checkpointing=True,
    eval_steps=1,
    save_strategy='epoch',
    save_total_limit=1, # save fewer checkpoints to limit used space
    report_to="mlflow",  # log to mlflow
)

# Move the model to the appropriate device
device = 'mps'
model.to(device)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dd["train"],
    eval_dataset=dd["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.evaluate()

100%|██████████| 219/219 [03:12<00:00,  1.08it/s]python(11574) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(11575) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(11576) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
100%|██████████| 219/219 [03:13<00:00,  1.13it/s]


{'eval_loss': 1.0916937589645386,
 'eval_roc_auc': 0.8105376457696902,
 'eval_accuracy': 0.4520547945205479,
 'eval_runtime': 194.4391,
 'eval_samples_per_second': 1.126,
 'eval_steps_per_second': 1.126}

In [26]:
trainer.train()

  0%|          | 1/438 [00:18<2:16:27, 18.74s/it]

{'loss': 1.1243, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.0}


  0%|          | 2/438 [00:35<2:06:34, 17.42s/it]

{'loss': 1.1132, 'learning_rate': 2.0000000000000002e-07, 'epoch': 0.01}


  1%|          | 3/438 [00:51<2:03:38, 17.05s/it]

...

{'loss': 1.0636, 'learning_rate': 6.000000000000001e-07, 'epoch': 0.03}


  2%|▏         | 7/438 [02:04<2:08:35, 17.90s/it]

{'loss': 1.0766, 'learning_rate': 7.000000000000001e-07, 'epoch': 0.03}


  2%|▏         | 8/438 [02:21<2:06:00, 17.58s/it]

{'loss': 1.1004, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.04}


In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model()

In [ ]:
from transformers import pipeline

pipe=pipeline('audio-classification',model=model_name,device=0)

In [ ]:
# Drum example
audio,rate=torchaudio.load('/kaggle/input/musical-instruments-sound-dataset/Train_submission/Train_submission/DRUM_SOUND (48).wav')
transform=torchaudio.transforms.Resample(rate,RATE_HZ)
audio=transform(audio).numpy().reshape(-1)[:MAX_LENGTH]
# make a classification pipeline
pipe(audio)

In [ ]:
# Guitar example
audio,rate=torchaudio.load('/kaggle/input/musical-instruments-sound-dataset/Test_submission/Test_submission/rock_6_100BPM.wav')
transform=torchaudio.transforms.Resample(rate,RATE_HZ)
audio=transform(audio).numpy().reshape(-1)[:MAX_LENGTH]
print(audio.shape)
# make a classification pipeline
pipe(audio)

In [ ]:
from IPython.display import Audio
Audio(audio,rate=RATE_HZ)

# Send model to Huggingface

In [ ]:
# finally, save the model to Huggingface
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
repo_id = f"dima806/{model_name}"
try:
    api.create_repo(repo_id)
except:
    print(f"Repo {repo_id} already exists")

In [ ]:
api.upload_folder(
    folder_path=model_name,
    path_in_repo = ".",
    repo_id=repo_id,
    repo_type="model"
)